# LangGraph Tutorial: Graph Construction

## Objective
Build the agentic workflow that connects the LLM to tools, creating a complete agent that can reason and act.

## What You'll Learn
1. Core LangGraph concepts: State, Nodes, Edges
2. How to bind tools to an LLM
3. Creating the Agent node (LLM decision-maker)
4. Creating the Router function (conditional logic)
5. Using the prebuilt ToolNode
6. Compiling and visualizing the graph

## Prerequisites
- Completed: Notebooks 01-04 (Setup, Tools basics, Currency Converter, EMI Calculator)
- Understanding of how tools work with `.invoke()`

---

## Section 1: The Agent Architecture

### What We're Building

```
START → Agent (LLM) → Router → [Tools or END]
           ↑                        ↓
           └────────────────────────┘
```

### Reference Point: LangGraph Core Concepts

| Concept | Description | Our Implementation |
|---------|-------------|-------------------|
| **State** | Data that flows through the graph | `MessagesState` (list of messages) |
| **Node** | A function that processes state | `agent` (LLM), `tools` (ToolNode) |
| **Edge** | Connection between nodes | START→agent, tools→agent |
| **Conditional Edge** | Dynamic routing based on state | agent→tools OR agent→END |

### The ReAct Pattern

Our agent follows the **ReAct** (Reasoning + Acting) pattern:

1. **Reason:** LLM analyzes the query and available tools
2. **Act:** LLM decides to call a tool (or respond directly)
3. **Observe:** Tool result is added to message history
4. **Repeat:** LLM reasons again with new information

This cycle continues until the LLM has enough information to respond.

---

## Section 2: Setup and Imports

In [ ]:
# Environment setup
import os
from dotenv import load_dotenv

load_dotenv("../../.env")  # Adjust path as needed
print("✅ Environment loaded")

In [ ]:
# LangGraph and LangChain imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from typing import Literal

print("✅ All imports successful")

In [ ]:
# Mermaid helper for graph visualization
def render_mermaid(diagram_code, width=400):
    """Render Mermaid diagrams using mermaid.ink service."""
    from IPython.display import Image, display
    import base64
    
    graphbytes = diagram_code.encode('utf-8')
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode('ascii')
    url = f'https://mermaid.ink/img/{base64_string}'
    display(Image(url=url, width=width))

print("✅ Visualization helper defined")

### Reference Point: Key Imports Explained

| Import | Purpose |
|--------|--------|
| `StateGraph` | Creates the graph structure |
| `MessagesState` | Prebuilt state with `messages` list |
| `START`, `END` | Special nodes for graph entry/exit |
| `ToolNode` | Prebuilt node that executes tools |
| `Literal` | Type hint for router return values |

---

## Section 3: Define the Tools

We'll recreate the tools from previous notebooks. In a real project, you'd import these from a shared module.

In [ ]:
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    
    Args:
        amount: The amount to convert
        from_currency: Source currency code (USD, EUR, GBP, INR, JPY)
        to_currency: Target currency code (USD, EUR, GBP, INR, JPY)
    
    Returns:
        A string with the conversion result including the exchange rate
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency {from_currency}"
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency {to_currency}"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

print("✅ currency_converter tool defined")

In [ ]:
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    
    Args:
        principal: The loan amount (must be greater than 0)
        annual_interest_rate: Annual interest rate as percentage (e.g., 8.5 for 8.5%)
        tenure_months: Loan tenure in months (must be greater than 0)
        currency: Currency code for display (USD, EUR, GBP, INR, JPY)
    
    Returns:
        A string with EMI calculation details
    """
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ emi_calculator tool defined")

In [ ]:
# Create tools list
tools = [currency_converter, emi_calculator]

print("\nTools Available:")
print("=" * 50)
for t in tools:
    print(f"  • {t.name}")

---

## Section 4: Initialize LLM with Tools

The key step: **bind tools to the LLM**. This tells the LLM what tools are available and how to call them.

In [ ]:
# Create base LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools")
print(f"   Model: gemini-2.0-flash")
print(f"   Tools bound: {len(tools)}")

### Reference Point: What Does `.bind_tools()` Do?

When you call `llm.bind_tools(tools)`, it:

1. **Extracts schemas** from each tool (name, description, parameters)
2. **Adds tool definitions** to every LLM request
3. **Enables tool_calls** in the LLM's response format

```python
# Without tools:
llm.invoke("Convert 100 USD to EUR")
# Returns: AIMessage(content="I don't have access to exchange rates...")

# With tools:
llm_with_tools.invoke("Convert 100 USD to EUR")
# Returns: AIMessage(tool_calls=[{name: "currency_converter", args: {...}}])
```

> **Key Insight:** The LLM doesn't execute tools—it only requests them. The `tool_calls` attribute contains the tool name and arguments the LLM wants to use.

---

## Section 5: Define the Agent Node

The **Agent Node** is a function that:
1. Receives the current state (message history)
2. Invokes the LLM
3. Returns the LLM's response to be added to state

In [ ]:
def call_llm(state: MessagesState):
    """
    Agent node that invokes the LLM.
    
    The LLM analyzes the conversation and decides to either:
    1. Call tools (returns AIMessage with tool_calls)
    2. Provide final response (returns AIMessage with content)
    
    Args:
        state: Current graph state containing message history
        
    Returns:
        Dictionary with "messages" key containing the LLM response
    """
    # Get all messages from state
    messages = state["messages"]
    
    # Invoke LLM with full conversation history
    response = llm_with_tools.invoke(messages)
    
    # Return response to be added to state
    # LangGraph automatically appends this to state["messages"]
    return {"messages": [response]}

print("✅ Agent node (call_llm) defined")

### Reference Point: Node Function Pattern

All LangGraph node functions follow this pattern:

```python
def my_node(state: StateType) -> dict:
    # 1. Read from state
    data = state["key"]
    
    # 2. Process
    result = do_something(data)
    
    # 3. Return updates to state
    return {"key": [result]}  # For MessagesState, values are appended
```

**Important:** With `MessagesState`, returned messages are **appended** to the existing list, not replaced.

---

## Section 6: Define the Router Function

The **Router** examines the LLM's response and decides where to go next:
- If LLM wants to use tools → go to `tools` node
- If LLM is done → go to `END`

In [ ]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """
    Router that determines the next node.
    
    Checks the last message in state:
    - If it has tool_calls → route to "tools" node
    - Otherwise → route to END (finish)
    
    Args:
        state: Current graph state
        
    Returns:
        Either "tools" string or END constant
    """
    # Get the last message (most recent LLM response)
    last_message = state["messages"][-1]
    
    # Check if LLM wants to call tools
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    
    # No tool calls = LLM is done, end the conversation
    return END

print("✅ Router function (should_continue) defined")

### Reference Point: Router Decision Logic

```
Last Message Analysis:
┌─────────────────────────────────────────────────────────┐
│ AIMessage                                               │
│   ├── content: "Here's your answer..."                  │
│   └── tool_calls: []  ──────────────────→ Route to END  │
└─────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────┐
│ AIMessage                                               │
│   ├── content: ""                                       │
│   └── tool_calls: [{name: "currency_converter", ...}]   │
│                    └────────────────────→ Route to TOOLS│
└─────────────────────────────────────────────────────────┘
```

The router doesn't make decisions—it just reads what the LLM decided.

---

## Section 7: Build the Graph

Now we connect all components into an executable workflow.

In [ ]:
print("Building StateGraph...")
print("=" * 50)

# Step 1: Initialize graph with state schema
workflow = StateGraph(MessagesState)
print("\n1. Created StateGraph with MessagesState")

# Step 2: Add nodes
workflow.add_node("agent", call_llm)          # Our LLM decision-maker
workflow.add_node("tools", ToolNode(tools))   # Prebuilt tool executor
print("2. Added nodes: 'agent', 'tools'")

# Step 3: Add entry point
workflow.add_edge(START, "agent")  # Always start with agent
print("3. Added edge: START → agent")

# Step 4: Add conditional routing from agent
workflow.add_conditional_edges(
    "agent",           # From this node
    should_continue,   # Use this function to decide
    {
        "tools": "tools",  # If returns "tools", go to tools node
        END: END            # If returns END, finish
    }
)
print("4. Added conditional edges: agent → [tools OR END]")

# Step 5: Add edge from tools back to agent (the loop)
workflow.add_edge("tools", "agent")
print("5. Added edge: tools → agent (creates the loop)")

# Step 6: Compile into executable app
app = workflow.compile()
print("\n" + "=" * 50)
print("✅ Graph compiled successfully!")

### Reference Point: Graph Building Steps

| Step | Method | Purpose |
|------|--------|--------|
| 1 | `StateGraph(MessagesState)` | Create graph with state schema |
| 2 | `add_node(name, function)` | Register processing functions |
| 3 | `add_edge(START, node)` | Define entry point |
| 4 | `add_conditional_edges()` | Define dynamic routing |
| 5 | `add_edge(node1, node2)` | Define fixed transitions |
| 6 | `compile()` | Create executable application |

---

## Section 8: Visualize the Graph

LangGraph can generate a Mermaid diagram of your workflow.

In [ ]:
# Get and display the Mermaid diagram
mermaid_diagram = app.get_graph().draw_mermaid()

print("Graph Visualization:")
print("=" * 50)
render_mermaid(mermaid_diagram, width=500)

In [ ]:
# Also print the raw Mermaid code
print("Mermaid Diagram Code:")
print("=" * 50)
print(mermaid_diagram)

### Reference Point: Understanding the Graph Structure

```
┌─────────┐
│  START  │
└────┬────┘
     │
     ▼
┌─────────┐     tool_calls?      ┌─────────┐
│  agent  │─────────YES─────────▶│  tools  │
└────┬────┘                      └────┬────┘
     │                                │
     │ NO (no tool_calls)             │
     ▼                                │
┌─────────┐                           │
│   END   │◀──────────────────────────┘
└─────────┘
```

**The Loop:** After tools execute, control returns to `agent`, which can:
- Call more tools (continue looping)
- Provide final answer (exit to END)

---

## Section 9: Understanding the Execution Flow

### Complete Message Flow Example

```
User: "Convert 100 USD to EUR"

┌──────────────────────────────────────────────────────────────────┐
│ STEP 1: START → agent                                           │
│   State: [HumanMessage("Convert 100 USD to EUR")]               │
│   Agent receives query, decides to call tool                    │
│   Output: AIMessage(tool_calls=[{currency_converter, args}])    │
└──────────────────────────────────────────────────────────────────┘
                              ↓
┌──────────────────────────────────────────────────────────────────┐
│ STEP 2: agent → tools (router sees tool_calls)                  │
│   ToolNode executes currency_converter.invoke({...})            │
│   Output: ToolMessage(content="Conversion Result: ...")         │
└──────────────────────────────────────────────────────────────────┘
                              ↓
┌──────────────────────────────────────────────────────────────────┐
│ STEP 3: tools → agent                                           │
│   Agent sees tool result, formulates final answer               │
│   Output: AIMessage(content="100 USD equals 92 EUR...")         │
└──────────────────────────────────────────────────────────────────┘
                              ↓
┌──────────────────────────────────────────────────────────────────┐
│ STEP 4: agent → END (router sees no tool_calls)                 │
│   Execution complete!                                           │
└──────────────────────────────────────────────────────────────────┘
```

### Reference Point: ToolNode Behavior

The prebuilt `ToolNode` automatically:

| Feature | Description |
|---------|-------------|
| **Extracts tool calls** | Reads `tool_calls` from the last AIMessage |
| **Matches tools** | Finds the right tool by name |
| **Executes tools** | Calls `.invoke()` with the provided args |
| **Handles parallel calls** | Uses ThreadPoolExecutor for multiple tools |
| **Returns ToolMessages** | Creates properly formatted responses |

> **Key Point:** You don't write tool execution logic—`ToolNode` handles everything!

---

## Section 10: Test the Graph

Let's verify the graph works with a simple test.

In [ ]:
# Create test state with a user query
test_state = {
    "messages": [HumanMessage(content="Convert 100 USD to EUR")]
}

print("Test Query: Convert 100 USD to EUR")
print("=" * 70)

In [ ]:
# Execute the graph
result = app.invoke(test_state)

print("\nExecution Complete!")
print("=" * 70)
print(f"Total messages in result: {len(result['messages'])}")
print("\nFinal Response:")
print("-" * 70)
print(result["messages"][-1].content)

In [ ]:
# Examine all messages in the execution
print("\nMessage History:")
print("=" * 70)

for i, msg in enumerate(result["messages"]):
    msg_type = type(msg).__name__
    print(f"\n[{i+1}] {msg_type}")
    print("-" * 40)
    
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        print(f"    Tool Calls: {len(msg.tool_calls)}")
        for tc in msg.tool_calls:
            print(f"      • {tc['name']}({tc['args']})")
    elif hasattr(msg, "content") and msg.content:
        content_preview = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
        print(f"    Content: {content_preview}")

---

## Summary

In this notebook, you learned:

| Concept | Key Takeaway |
|---------|-------------|
| **State** | `MessagesState` holds conversation history |
| **bind_tools()** | Enables LLM to request tool execution |
| **Agent Node** | Invokes LLM and returns response |
| **Router** | Directs flow based on `tool_calls` presence |
| **ToolNode** | Prebuilt node that executes tools automatically |
| **The Loop** | tools → agent enables multi-step reasoning |

## What You Built

```
✅ Agent node (LLM decision-maker)
✅ Router function (conditional logic)
✅ Tools node (executes tools)
✅ Graph with cycle (agent ↔ tools loop)
✅ Compiled executable application
```

## Next Steps

In the following notebooks, we'll explore different execution patterns:

| Notebook | Topic | Pattern |
|----------|-------|--------|
| **06** | Single Tool Execution | One tool call per query |
| **07** | Parallel Execution | Multiple independent tools |
| **08** | Sequential Execution | Dependent tool chains |
| **09** | Conversational Context | Multi-turn conversations |

---